In [1]:
from time import time
from sage.coding.guruswami_sudan.interpolation import gs_interpolation_lee_osullivan
from sage.coding.guruswami_sudan.gs_decoder import alekhnovich_root_finder, roth_ruckenstein_root_finder

class RS:
    def __init__(self,q,n,k, K, g):
        if n > q :
            print("Error, n must be equal or less than q")
            return -1
        self.q = q
        self.n = n
        self.k = k
        self.d = n - k + 1
        self.t = floor((self.d - 1)/2.0)
        self.K = K
        self.g = g
        self.x = [g^i for i in range(n-1)]
        self.x.append(0)
     
    def encode(self,M):
        return vector(self.K,[sum([self.x[j]^(self.k-i-1)*M[i] for i in range(self.k)]) for j in range(self.n)])
        
    def distance_min(self):
        info_space = VectorSpace(self.K,self.k)
        w_min = self.n

        for M in info_space:
            w_tmp = self.encode(M).hamming_weight()
            if w_tmp > 0 and w_tmp<w_min:
                w_min = w_tmp
                
        return w_min
    
    def decode_BW(self,U):
        M = Matrix(self.K, self.n, self.n + 1) 
        for i in range(self.n):
            for j in range(self.t + 1):
                M[i,j] = U[i]*(self.x[i])^j
            for j in range(self.n - self.t):
                M[i,self.t+1+j] = -(self.x[i])^(j)
        #print(M)
        
        coefs = M.right_kernel().basis_matrix().rows()[0]
        R.<X> = PolynomialRing(self.K)
        A = 0
        B = 0
        tmp = 1
        for i in range(self.t + 1):
            A += coefs[i]*tmp
            tmp *= X
        tmp = 1
        for i in range(self.n - self.t):
            B += coefs[self.t + i + 1] * tmp
            tmp *= X
        r = B//A
        coefs = r.list() # https://ask.sagemath.org/question/26907/get-the-coefficients-from-the-polynomial/
        
        while len(coefs)<self.k:
            coefs.append(0)
        coefs.reverse()
        return vector(self.K, coefs)
    
    def decode_BM(self, u):
        pol.<X> = PolynomialRing(self.K)
        points = []
        Pi = 1
        for i in range(self.n):
            points.append((self.x[i],u[i]))
            Pi = Pi * (X - self.x[i])
        U = pol.lagrange_polynomial(points)
        #print(U)
        C0, A0, B0 = 1, 0 , Pi
        C1, A1, B1 = 0, 1 , U
        while B1.degree() >= self.n - self.t:
            B2 = B0 % B1
            Q = B0 // B1
            A2 = A0 - Q * A1
            C2 = C0 - Q * C1
            A0, B0, C0 = A1, B1, C1
            A1, B1, C1 = A2, B2, C2
        r = B1//A1
        coefs = r.list() # https://ask.sagemath.org/question/26907/get-the-coefficients-from-the-polynomial/
        
        while len(coefs)<self.k:
            coefs.append(0)
        coefs.reverse()
        return vector(self.K, coefs)
        

    def poly_to_coefs(self, poly):
        coefs = poly.list() # https://ask.sagemath.org/question/26907/get-the-coefficients-from-the-polynomial/
        while len(coefs)<self.k:
            coefs.append(0)
        coefs.reverse()
        return coefs

    def decode_GuSu(self, u, radius, multiplicity):
        points = []
        l = ceil(multiplicity*(self.n - radius)/(self.k - 1)) - 1 
        for i in range(self.n):
            points.append((self.x[i],u[i]))
        Q = gs_interpolation_lee_osullivan(points, radius, (multiplicity, l), self.k - 1 )
        #F = alekhnovich_root_finder(Q, maxd = self.k - 1 )
        F = roth_ruckenstein_root_finder(Q, maxd = self.k - 1 )
        if len(F) == 0:
            print("Error, no code found")
            return 0
        if len(F) == 1:
            return self.poly_to_coefs(F[0])
        else :
            # search the nearest solution to u
            uu = vector(self.K, u)
            candidat = 0
            min_w = self.n
            for i in range(len(F)):
                coefs = self.poly_to_coefs(F[i])
                encoded = self.encode(coefs)
                encoded = vector(self.K, encoded)
                print(encoded)
                diff = (uu - encoded).hamming_weight()
                if diff < min_w :
                    candidat = i
                    min_w = diff
            return self.poly_to_coefs(F[candidat])   
    
    def decode_Su(self, u, radius):
        return decode_GuSu(self, u, radius, 1)

In [2]:
def canal(x, nb_err, K):
    n = len(x)
    I = Subsets(n,nb_err).random_element()
    y = []
    for i in range(n):
        y.append(x[i])
        if i+1 in I:
            err = 0
            while err == 0:
                err = K.random_element()
            y[i] += err
    return vector(K,y)
 

# Test Berlekamp Welch

In [3]:
m = 8
q = 2^m
n = 200
k = 150
K.<g> = GF(q)
R = RS(q,n,k,K,g)

mess = vector(K, [R.K.random_element() for i in range(R.k)])
#print("Information : ", mess)

m = R.encode(mess)
#print("Code word   : ", m)
U = canal(m, R.t, K)
#print("Received    : ", U)
t1 = time()
m1 = R.decode_BW(U)
t2 = time()
#print("decoded information:", m1)
print("Well decoded :", m1 == mess)
print("time to decode :", t2-t1 ," seconds")



Well decoded : True
time to decode : 0.6034340858459473  seconds


# Test Berlekamp Massey

In [4]:
m = 8
q = 2^m
n = 200
k = 150
K.<g> = GF(q)
R = RS(q,n,k,K,g)

mess = vector(K, [R.K.random_element() for i in range(R.k)])
#print("Information : ", mess)

m = R.encode(mess)
#rint("Code word   : ", m)
U = canal(m, R.t, K)
#print("Received    : ", U)
t1 = time()
m1 = R.decode_BM(U)
t2 = time()
#print("decoded information:", m1)
print("Well decoded :", m1 == mess)
print("time to decode :", t2-t1 ," seconds")

Well decoded : True
time to decode : 1.6989481449127197  seconds


# Test Gurusuami Sudan

In [37]:
m = 8
base = 2
q = base^m
n = 13
k = 10
K.<g> = GF(q)



grs = RS(q, n, k, K, g)
mess = vector(K, [K.random_element() for i in range(k)])
#print("Message : ", mess)
c = grs.encode(mess)
#print("Encoded message :", c)
u = canal(c, grs.t + 1,  K)
#print("Code received : ", u)
t1 = time()
m = grs.decode_GuSu( u, grs.t + 2, n) 
t2 = time()
m = vector(K, m)
#print("Message decoded :", m)
print("Well decoded :", m == mess)
print("Time to decode : ", t2 - t1, " seconds")

Well decoded : True
Time to decode :  151.5898916721344  seconds
